# 개체명 인식(Named Entity Recognition)


* 개체명 인식은 텍스트에서 이름을 가진 개체를 인식하는 기술      
* 가령, '철수와 영희는 밥을 먹었다'에서 이름과 사물을 추출하는 개체명 인식 모델 결과 

  철수 - 이름    
  영희 - 이름    
  밥 - 사물

## 개체명 인식 - NLTK

* https://wikidocs.net/30682

* `nltk` 라이브러리에서는 미리 학습된 개체명 인식 모델을 제공

### 라이브러리 준비

In [1]:
import nltk

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

### 토큰화 및 품사 태깅

In [2]:
from nltk import word_tokenize, pos_tag, ne_chunk

sentence = 'James is working at Disney in Londen'
sentence = pos_tag(word_tokenize(sentence))

print(sentence)

[('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('Disney', 'NNP'), ('in', 'IN'), ('Londen', 'NNP')]


### 개체명 인식

In [3]:
sentence = ne_chunk(sentence)

print(sentence)

# person
# organizatino 기관
# gpe

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Disney/NNP)
  in/IN
  (GPE Londen/NNP))


## 개체명 인식 - LSTM

* https://wikidocs.net/24682

* 사용자가 제공되고 있는 개체명 인식 모델과는 다른 개체명을 정의해 사용하는 것이 필요할 수 있음
* 직접 개체명 인식 모델을 구성해 학습하고 사용할 수 있음

### 라이브러리 준비

In [4]:
import numpy as np
import urllib.request


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


### 데이터 준비

* 공개된 개체명 인식 데이터셋을 이용
  + https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt
* 해당 데이터는 단어-개체명 형식으로 이루어져 있으므로 이를 가공해 데이터셋을 생성

In [20]:
tagged_sentences = []
sentence = []


with urllib.request.urlopen('https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt') as f:
    for line in f:
        line = line.decode('utf-8')
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == '\n':   # docstart거나 개행문자일때
            if len(sentence) > 0:
                tagged_sentences.append(sentence)
                sentence = []
            continue
        splits = line.strip().split(' ')    # space bar 기준 
        word = splits[0].lower()    # 소문자변환
        sentence.append([word,splits[-1]])  # 단어랑 개체명태깅을 pair로

print(len(tagged_sentences))
print(tagged_sentences[0])

# 이런 식으로 저장되어잇대


14041
[['eu', 'B-ORG'], ['rejects', 'O'], ['german', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['british', 'B-MISC'], ['lamb', 'O'], ['.', 'O']]


### 데이터 전처리

* 단어와 개체명 태그를 분리해서 데이터를 구성

In [21]:
sentences, ner_tags = [], []

for tagged_sentence in tagged_sentences:
    sentence, tag_info = zip(*tagged_sentence)     # 가져온것드른 tag_inof에, 단어들은 sentence에 넣는거지
    sentences.append(list(sentence))
    ner_tags.append(list(tag_info))

* 정제 및 빈도 수가 높은 상위 단어들만 추출하기 위해 토큰화 작업

In [22]:
max_words = 4000
src_tokenizer = Tokenizer(num_words = max_words, oov_token = 'OOV')
src_tokenizer.fit_on_texts(sentences)

tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(ner_tags)



In [23]:
vocab_size = max_words
tag_size = len(tar_tokenizer.word_index) + 1


print(vocab_size)   # 4000개로 제한했으니까
print(tag_size)

4000
10


* 데이터를 학습에 활용하기 위해 데이터를 배열로 변환
* 해당 작업은 토큰화 툴의 `texts_to_sequences()`를 통해 수행

In [24]:
print(sentences[0])
print(X_train[0])

['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
[1299   15    1   36 1348    6  627  139    4   24 2648    1  179   16
  469 1005  611  443    9 3283  635    1  481    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]


In [25]:
X_train = src_tokenizer.texts_to_sequences(sentences)
y_train = tar_tokenizer.texts_to_sequences(ner_tags)

* 학습에 투입할 때는 동일한 길이를 가져야 하므로, 지정해둔 최대 길이에 맞춰 모든 데이터를 동일한 길이로 맞춰줌
* 일반적으로 길이를 맞출 때는 모자란 길이만큼 0을 추가


In [26]:
# 페딩을 해주라는 말

max_len = 70
X_train = pad_sequences(X_train, padding = 'post', maxlen= max_len)    # 이번엔 패딩을 뒤로할게
y_train = pad_sequences(y_train, padding='post', maxlen = max_len)


In [27]:
print(sentences[0])
print(X_train[0])

['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
[ 989    1  205  629    7 3939  216    1    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]


* 훈련, 실험 데이터 분리 및 원 핫 인코딩을 시행

In [28]:
# 원핫 인코등으로 변경해서 to categorical로 해야지
# 일단 split부터 해보자

X_train,X_test, y_train, y_test  = train_test_split(X_train, y_train, test_size = 0.2, random_state = 111)
y_train = to_categorical(y_train, num_classes = tag_size)
y_test = to_categorical(y_test, num_classes = tag_size)


* 최종적으로 생성된 데이터셋의 크기는 다음과 같음

In [29]:
print(X_train.shape)     #훈령용
print(y_train.shape)    # tag의 갯수포함이지
print(X_test.shape)
print(y_test.shape)

(11232, 70)
(11232, 70, 10)
(2809, 70)
(2809, 70, 10)


### 모델 구축 및 학습

* 모델 구축에는 `keras`를 이용
* 해당 작업에 필요한 함수들을 추가로 import

In [30]:
from keras.models import Sequential
from keras.layers import Dense,Embedding, LSTM, Bidirectional, TimeDistributed
from keras. optimizers import Adam



모델의 구성

1. 입력을 실수 벡터로 임베딩
2. 양방향 LSTM 구성
3. Dense layer를 통한 각 태그에 속할 확률 예측

`TimeDistributed`는 상위 layer의 출력이 step에 따라 여러 개로 출력되어 이를 적절하게 분배해주는 역할

In [35]:
model = Sequential()
model.add(Embedding(input_dim= vocab_size, output_dim=128, input_length= max_len, mask_zero = True))
model.add(Bidirectional(LSTM(256, return_sequences = True)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))   


# 사실은 다중분류 문제랑 같은ㄱ지

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 70, 128)           512000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 70, 512)           788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 70, 10)            5130      
Total params: 1,305,610
Trainable params: 1,305,610
Non-trainable params: 0
_________________________________________________________________


* 모델 컴파일 및 학습 진행, 평가

In [36]:
model.compile(loss = 'categorical_crossentropy',
              optimizer =Adam(learning_rate=0.001),
              metrics = ['accuracy'])

model.fit(X_train, y_train,
          batch_size= 128,
          epochs = 3,
          validation_data = (X_test, y_test))

# validation data를 test data로 썻기때문에 val acc는 상관없지

Epoch 1/3
88/88 [==============================] - 120s 1s/step - loss: 0.1880 - accuracy: 0.8226 - val_loss: 0.1322 - val_accuracy: 0.8333
Epoch 2/3
88/88 [==============================] - 116s 1s/step - loss: 0.1016 - accuracy: 0.8532 - val_loss: 0.0781 - val_accuracy: 0.8826
Epoch 3/3
88/88 [==============================] - 118s 1s/step - loss: 0.0655 - accuracy: 0.9036 - val_loss: 0.0544 - val_accuracy: 0.9213


In [37]:
model.evaluate(X_test, y_test)

88/88 [==============================] - 13s 148ms/step - loss: 0.0544 - accuracy: 0.9213


[0.05436350032687187, 0.9212540984153748]

### 학습한 모델을 통한 예측

* 예측을 확인하기 위해서 인덱스를 단어로 변환해줄 사전이 필요
* 사전은 토큰화 툴의 사전을 이용

In [39]:
idx2word = src_tokenizer.index_word
idx2ner = tar_tokenizer.index_word

idx2ner[0] = 'PAD'


* 예측 시각화

In [48]:
# 시각화라기보단 text 출력이지

i = 30  # 10번 확인해볼게
y_predicted = model.predict(np.array([X_test[i]]))  # 예측된 y값
y_predicted = np.argmax(y_predicted, axis= -1)  # 가장 큰값으로
true = np.argmax(y_test[i], -1 )    # 정답 y는 개체명이 되겠지, 실제 i번에 해당하는 개체명 가져오기
                                    # catogorical data인데 원핫인코딩 되어있는걸 정수로 바꿔주는거지

print('{:15}|{:5}|{}'.format('단어', '실제값','예측값'))
print('-'*34)

for w , t , pred in zip(X_test[i], true, y_predicted[0]):   #word, true, predict값 // i번째 실제 단어, true= 실제값, y_predcted[0]= 예측한 값
    if w != 0:  # 아까 pad값 줬잖아
        print('{:17}: {:7} {}'.format(idx2word[w], idx2ner[t].upper(),idx2ner[pred].upper()))   # 0인거 아닌거는 일단 다출력// 일단 index로 받아오니까 word로 바꿔주두록 아까 만들었던 걸 사용


#  thorpe 하나 틀렸네

단어             |실제값  |예측값
----------------------------------
graham           : B-PER   B-ORG
thorpe           : I-PER   B-ORG
3                : O       O
5                : O       O
0                : O       O
OOV              : O       O
77               : O       O
OOV              : O       O
